In [ ]:
from _collections import defaultdict
import random
import re

def get_usp10(seq, location):
    left = location - 1
    right = len(seq) - location
    if left >= 10 and right > 10:
        usp = seq[location - 10: location + 11]
    elif left < 10 and right > 10:
        usp = seq[0:location + 11].rjust(21, '*')
    elif left >= 10 and right <= 10:
        usp = seq[location - 10: -1].ljust(21, '*')
    else:
        usp = seq[0:location + 1].rjust(11, '*') + seq[location + 1:-1].ljust(10, '*')
    return usp

# ''' 去冗余后的fasta读入，peplist蛋白列名 3836
fasta = open('2.rmdup_new.fasta', mode='r')
fasta = fasta.read().split('>')[1:]
peplist = []
for fast in fasta:
    pepname = fast.split('\n')[0]
    peplist.append(pepname)
print(len(peplist))

# ''' 读入蛋白id及位点 3836
txt = open('pre_training_site.txt', mode='r')
txt = txt.readlines()
Site_dict = defaultdict(list)
for line in txt:
    line = line.rstrip().split('\t')
    pep = line[0]
    site_str = line[1]
    if site_str.isdigit():  # 检查位置信息是否都是数字字符
        site = int(site_str)
        if pep not in Site_dict:
            Site_dict[pep].append(site)
        elif site not in Site_dict[pep]:
            Site_dict[pep].append(site)
        else:
            continue
    else:
        print(f"Invalid site value: {site_str}")
print(len(Site_dict.keys()))

# ''' 读入蛋白id及序列 3836
Seq_dict = {}
with open('pre_training_sequences.txt', mode='r') as fasta_file:
    lines = fasta_file.readlines()

    current_pep = None
    current_seq = ''

    for line in lines:
        line = line.rstrip()
        if line.startswith('>'):
            # If the line starts with '>', it's a header line indicating a new sequence
            if current_pep is not None:
                Seq_dict[current_pep] = current_seq
                current_seq = ''
            # Extract the peptide ID without '>'
            matches = re.findall(r'\|(.*?)\|', line)  # Assuming '|' are your desired symbols
            if matches:  # Check if matches were found
                current_pep = matches[0]  # Take the first match
        else:
            # Concatenate sequence lines
            current_seq += line

    # Add the last sequence to the dictionary
    if current_pep is not None:
        Seq_dict[current_pep] = current_seq
print(len(Seq_dict.keys()))

# ''' 筛选K非kcr位点
neg_Site_dict = defaultdict(list)
for pepname in Seq_dict.keys():
    for i, aa in enumerate(Seq_dict[pepname]):
        site = i + 1
        if aa == 'K' and site not in Site_dict[pepname]:
            if pepname not in neg_Site_dict.keys():
                neg_Site_dict[pepname].append(site)
            elif site not in neg_Site_dict[pepname]:
                neg_Site_dict[pepname].append(site)

# 获取 Site_dict 和 Seq_dict 的键集合
site_dict_keys = set(Site_dict.keys())
seq_dict_keys = set(Seq_dict.keys())

# 获取 Site_dict 中存在而 Seq_dict 中不存在的键
not_matched_keys = site_dict_keys - seq_dict_keys

if not_matched_keys:
    print("Site_dict 中存在而 Seq_dict 中不存在的键:")
    for key in not_matched_keys:
        print(key)
else:
    print("Site_dict 和 Seq_dict 中的键完全匹配。")

#'''pre_training有12242个pos_usp10
pos_usp10 = {}
for pepname in Site_dict.keys():
    if pepname in Seq_dict:
        for site in Site_dict[pepname]:
            pep_site = pepname + '_' + str(site)
            usp10 = get_usp10(Seq_dict[pepname], site - 1)
            pos_usp10[pep_site] = usp10
    else:
        print(f"Key {pepname} not found in Seq_dict.")
pos_len = len(pos_usp10.keys())
print(pos_len)

#'''创建文本保存pos_usp10
pos_set = open('pos_Kcr.txt', mode='w')
for ps in pos_usp10.keys():
    pos_set.write(ps + '\t' + pos_usp10[ps] + '\n')
pos_set.close()

#'''随机抽取同样数量的neg_usp10
neg_usp10 = {}
for pepname in neg_Site_dict.keys():
    for site in neg_Site_dict[pepname]:
        pep_site = pepname + '_' + str(site)
        usp10 = get_usp10(Seq_dict[pepname], site - 1)
        neg_usp10[pep_site] = usp10

neg_sample_list = random.sample(list(neg_usp10.keys()), pos_len)
neg_set = open('neg_Kcr.txt', mode='w')
for ns in neg_sample_list:
    neg_set.write(ns + '\t' + neg_usp10[ns] + '\n')
neg_set.close()

# 所有的K非kcr位点 142845
print(len(neg_usp10))
neg_k = open('neg_K.txt', mode='w')
for ns in neg_usp10:
    neg_k.write(ns + '\t' + neg_usp10[ns] + '\n')
neg_k.close()

In [ ]:
# 所有的K位点 
# 打开第一个文本文件并读取内容 142845
with open('neg_K.txt', 'r') as file1:
    content1 = file1.readlines()
print(len(content1))

# 打开第二个文本文件并读取内容 12242
with open('pos_Kcr.txt', 'r') as file2:
    content2 = file2.readlines()
print(len(content2))

# 将两个文本内容合并 155087
combined_content = content1 + content2
print(len(combined_content))

# 创建新的合并后的文本文件并写入内容
with open('background_K.txt', 'w') as combined_file:
    for line in combined_content:
        combined_file.write(line)

#'''读入被验证的位点，并构建pos_fkcr
Site_dict = defaultdict(list)
pos_fsites = open('reported.txt')
pos_fsites = pos_fsites.readlines()

for line in pos_fsites:
    line = line.rstrip().split('\t')
    pep = line[0]
    site = line[1]
    if site.isdigit():  # 检查位置信息是否都是数字字符
        site = int(site)
        if pep not in Site_dict:
            Site_dict[pep].append(site)
        elif site not in Site_dict[pep]:
            Site_dict[pep].append(site)
        else:
            continue
    else:
        print(f"Invalid site value: {site}")

fpos_usp10 = {}
for pepname in Site_dict.keys():
    if pepname in Seq_dict:
        for site in Site_dict[pepname]:
            pep_site = pepname + '_' + str(site)
            usp10 = get_usp10(Seq_dict[pepname], site - 1)
            fpos_usp10[pep_site] = usp10
    else:
        print(f"Key {pepname} not found in Seq_dict.")
fpos_len = len(fpos_usp10.keys())
file = open('pos_fKcr.txt', mode='w')
for s in fpos_usp10.keys():
    file.write(s + '\t' + fpos_usp10[s] + '\n')
file.close()

#'''构建neg_fkcr
# 读取要剔除的文件A
with open('pos_fKcr.txt', 'r') as fileA:
    lines_to_remove = set(fileA.read().splitlines())  #59
print(len(lines_to_remove))

# 读取参考文件B并转换为集合类型
with open('pos_Kcr.txt', 'r') as fileB:
    reference_lines = set(fileB.read().splitlines())  #12242
print(len(reference_lines))

# 进行集合操作，获取剔除后的内容
remaining_lines = reference_lines-lines_to_remove  # 12183
print(len(remaining_lines))

# 将剩余内容写入新文件
with open('neg_fKcr.txt', mode='w') as outfile:
    outfile.write('\n'.join(remaining_lines))

In [ ]:
# 本次实验位点 5735
Seq_dict = {}
Site_dict = defaultdict(list)
all_sites = open('experiment_sequence.txt')
all_sites = all_sites.readlines()
for line in all_sites:
    line = line.rstrip().split('\t')
    pep = line[0]
    sequence = line[1]
    site = int(line[2])
    if pep not in Seq_dict.keys():
        Seq_dict[pep] = sequence
    if pep not in Site_dict.keys():
        Site_dict[pep].append(site)
    elif site not in Site_dict[pep]:
        Site_dict[pep].append(site)
    else:
        continue

usp10_dict = {}
for pepname in Site_dict.keys():
    for site in Site_dict[pepname]:
        pep_site = pepname + '_' + str(site)
        usp10 = get_usp10(Seq_dict[pepname], site - 1)
        usp10_dict[pep_site] = usp10
file = open('experiment_sites.txt', mode='w')
for s in usp10_dict.keys():
    file.write(s + '\t' + usp10_dict[s] + '\n')
file.close()